Manejo de un DataFrame de Pandas como un array con tf.data --- 5:01 min
===

* Última modificación: Mayo 6, 2022 | [YouTube](https://youtu.be/SFYG_tF4T8w)

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import pandas as pd
import tensorflow as tf

Lectura de datos usando Pandas
---

In [2]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

In [3]:
csv_file = tf.keras.utils.get_file(
    "heart.csv",
    "https://storage.googleapis.com/download.tensorflow.org/data/heart.csv",
)

In [4]:
#
# Lecura
#
df = pd.read_csv(csv_file)
target = df.pop("target")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal


Extracción de características numéricas
---

In [5]:
numeric_feature_names = [
    "age",
    "thalach",
    "trestbps",
    "chol",
    "oldpeak",
]
numeric_features = df[numeric_feature_names]
numeric_features.head()

,age,thalach,trestbps,chol,oldpeak
0,63,150,145,233,2.3
1,67,108,160,286,1.5
2,67,129,120,229,2.6
3,37,187,130,250,3.5
4,41,172,130,204,1.4


Especificación del Modelo
---

In [6]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

#
# Ejemplo
#
normalizer(numeric_features.iloc[:3])

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.93383914,  0.03480718,  0.74578077, -0.2600867 ,  1.0680453 ],
       [ 1.3782105 , -1.7806165 ,  1.5923285 ,  0.7573879 ,  0.38022864],
       [ 1.3782105 , -0.87290466, -0.6651321 , -0.3368772 ,  1.3259765 ]],
      dtype=float32)>

In [7]:
def get_basic_model():
    model = tf.keras.Sequential(
        [
            normalizer,
            tf.keras.layers.Dense(10, activation="relu"),
            tf.keras.layers.Dense(10, activation="relu"),
            tf.keras.layers.Dense(1),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    return model

Manejo del DataFrame con tf.data
---

In [8]:
numeric_dataset = tf.data.Dataset.from_tensor_slices(
    (numeric_features, target),
)

for row in numeric_dataset.take(3):
    print(row)

(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 63. , 150. , 145. , 233. ,   2.3])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 108. , 160. , 286. ,   1.5])>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 129. , 120. , 229. ,   2.6])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [9]:
numeric_batches = numeric_dataset.shuffle(1000)
numeric_batches = numeric_batches.batch(BATCH_SIZE)

model = get_basic_model()
model.fit(numeric_batches, epochs=5)

Epoch 1/5
152/152 [==============================] - 1s 2ms/step - loss: 0.7444 - accuracy: 0.6766
Epoch 2/5
152/152 [==============================] - 0s 2ms/step - loss: 0.6545 - accuracy: 0.7261
Epoch 3/5
152/152 [==============================] - 0s 2ms/step - loss: 0.5689 - accuracy: 0.7261
Epoch 4/5
152/152 [==============================] - 0s 2ms/step - loss: 0.5089 - accuracy: 0.7261
Epoch 5/5
152/152 [==============================] - 0s 2ms/step - loss: 0.4834 - accuracy: 0.7261
